In [1]:
import src
import keras.backend as K
import os
import numpy as np
import sys
import re
import math
import io
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from  matplotlib.animation import FuncAnimation
from matplotlib import colors
from netCDF4 import Dataset
from IPython.display import clear_output
#data folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project/trainings')

sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/data')
#forces CPU usage
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #"" or "-1" for CPU, "0" for GPU
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13274176401914743264
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1508248780
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3685309914838679400
physical_device_desc: "device: 0, name: GeForce GT 740M, pci bus id: 0000:01:00.0, compute capability: 3.5"
]


In [2]:
past = 2
name = f"sgan_{past}-1_w"

Load dataset.

In [3]:
train, xval, test = src.load_datasets(past_frames=past)

Training data: (7500, 64, 64, 3)
Validation data: (1500, 64, 64, 3)
Test data: (1000, 64, 64, 3)


In [4]:
gan_train, gan_truth, gan_val, gan_val_truth, gan_test, gan_test_truth = src.split_datasets(
            train[:2000], xval, test, past_frames=past, augment=True)

Data augmentation.
Shape of training data:  (8000, 64, 64, 2) 
Shape of training truth:  (8000, 64, 64, 1) 
Shape of validation data:  (1500, 64, 64, 2) 
Shape of validation truth:  (1500, 64, 64, 1) 
Shape of test data:  (1000, 64, 64, 2) 
Shape of test truth:  (1000, 64, 64, 1)


Make discriminator labels.

In [3]:
batch_size=16

In [4]:
# Adversarial ground truths
real = -np.ones((batch_size, 1))
fake = np.ones((batch_size, 1))
#Generator ground truths
#real, fake = src.noisy_d_labels(real, fake)

Make generator but don't compile.

In [9]:
generator = src.unet((64, 64, past), dropout=0, batchnorm=False, kernel_size=7, feature_mult=1)
discriminator = src.spatial_discriminator(condition_shape=(64, 64, past), dropout = 0.25, batchnorm=True, wgan=True)
discriminator.compile(loss=src.wasserstein_loss,
                      optimizer=keras.optimizers.RMSprop(lr=0.00005))

frame_t = keras.layers.Input(shape=(64, 64, past))
generated = generator(frame_t)
score = discriminator([frame_t, generated])
discriminator.trainable = False
for l in discriminator.layers: l.trainable = False
loss_weights=[0,1]
combined = keras.models.Model(inputs=[frame_t], outputs=[generated, score])
combined.load_weights("C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project/models/"+name+"/sgan_2-1_w_model.h5")

ValueError: Layer #0 (named "model_13" in the current model) was found to correspond to layer model_13 in the save file. However the new layer model_13 expects 20 weights, but the saved weights have 56 elements.

In [17]:
combined.compile(loss=[src.custom_loss(loss="l1"), src.wasserstein_loss],
                 optimizer=keras.optimizers.RMSprop(lr=0.00005),
                 loss_weights=loss_weights)

***Using L1 loss.***


In [18]:
log = {"g_loss":[],
       "d_loss":[],
       "g_metric":[],
       "d_metric":[],
       "d_loss_real":[],
       "d_loss_fake":[],
       "d_test_real":[],
       "d_test_fake":[]}

Train x epochs.

In [19]:
epochs = 5000
# random seed
RND = 777
np.random.seed(RND)

In [ ]:
for epoch in range(epochs):
    
    if (epoch % 1000) < 25 or epoch % 500 == 0: # 25 times in 1000, every 500th
        d_iters = 100
    else:
        d_iters = 10
    for d_it in range(d_iters):
        discriminator.trainable = True
        for l in discriminator.layers: l.trainable = True
        idx = np.random.choice(gan_truth.shape[0], batch_size, replace=False)
        real_imgs = gan_truth[idx]
        training_batch = gan_train[idx]
        
        for l in discriminator.layers:
            weights = l.get_weights()
            weights = [np.clip(w, -0.01, 0.01) for w in weights]
            l.set_weights(weights)
        d_loss_real = discriminator.train_on_batch([training_batch, real_imgs], real)
        generated_imgs = generator.predict(training_batch) 
        d_loss_fake = discriminator.train_on_batch([training_batch, generated_imgs], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    discriminator.trainable = False 
    for l in discriminator.layers: l.trainable = False
    
    idx = np.random.choice(gan_truth.shape[0], batch_size, replace=False)
    training_batch = gan_train[idx]
    training_truth = gan_truth[idx]
    
    g_loss = combined.train_on_batch(training_batch, [training_truth, real])
    
    log["g_loss"].append(g_loss) #sum, obj, bce
    log["d_loss"].append(d_loss) #sum
    log["g_metric"].append(g_loss[1])
    #log["d_metric"].append(d_loss[1])
    log["d_loss_real"].append(d_loss_real)
    log["d_loss_fake"].append(d_loss_fake)
    
    print(f"\033[1m {epoch} [D loss: {d_loss}]\033[0m \n"+
          f"\033[1m {epoch} [G loss: {g_loss[0]}, G obj.: {g_loss[1]}, G wgan.: {g_loss[2]}]\033[0m \n"+
          f"\033[1m {epoch} [real loss: {d_loss_real}, fake loss: {d_loss_fake}]\033[0m")
    if (epoch%100 == 0 or epoch == epochs-1) and epoch > 0:
        src.sample_images(epoch, gan_test, gan_test_truth, past, generator)
        plot_training_curves(log, epoch, name, True)

 0 [D loss: 3.265789564466104e-05] 
 0 [G loss: -5.649941886076704e-05, G obj.: 0.42712098360061646, G wgan.: -5.649941886076704e-05] 
 0 [real loss: -8.717529999557883e-05, fake loss: 0.0001524910912849009]
 1 [D loss: 4.926981637254357e-05] 
 1 [G loss: -6.358361861202866e-05, G obj.: 0.45127883553504944, G wgan.: -6.358361861202866e-05] 
 1 [real loss: -5.761707143392414e-05, fake loss: 0.00015615670417901129]
 2 [D loss: 3.337911766720936e-05] 
 2 [G loss: -6.909283547429368e-05, G obj.: 0.4361945688724518, G wgan.: -6.909283547429368e-05] 
 2 [real loss: -5.907869490329176e-05, fake loss: 0.00012583693023771048]
 3 [D loss: 1.953593164216727e-05] 
 3 [G loss: -5.0490081775933504e-05, G obj.: 0.4431600570678711, G wgan.: -5.0490081775933504e-05] 
 3 [real loss: -8.82450258359313e-05, fake loss: 0.00012731688912026584]
 4 [D loss: -0.0004028014081995934] 
 4 [G loss: -0.00037602928932756186, G obj.: 0.42390504479408264, G wgan.: -0.00037602928932756186] 
 4 [real loss: -0.0007316403

 36 [D loss: -0.019839391112327576] 
 36 [G loss: -0.011536257341504097, G obj.: 0.4209196865558624, G wgan.: -0.011536257341504097] 
 36 [real loss: -0.015632135793566704, fake loss: -0.024046648293733597]
 37 [D loss: -0.016230793669819832] 
 37 [G loss: -0.013622907921671867, G obj.: 0.41101837158203125, G wgan.: -0.013622907921671867] 
 37 [real loss: -0.007807504385709763, fake loss: -0.0246540829539299]
 38 [D loss: -0.01437292154878378] 
 38 [G loss: -0.002826630836352706, G obj.: 0.37788403034210205, G wgan.: -0.002826630836352706] 
 38 [real loss: -0.005150930490344763, fake loss: -0.023594912141561508]
 39 [D loss: -0.017481280490756035] 
 39 [G loss: -0.012268561869859695, G obj.: 0.42208346724510193, G wgan.: -0.012268561869859695] 
 39 [real loss: -0.010390661656856537, fake loss: -0.024571899324655533]
 40 [D loss: -0.014947821386158466] 
 40 [G loss: -0.01191440038383007, G obj.: 0.40788912773132324, G wgan.: -0.01191440038383007] 
 40 [real loss: -0.005312548950314522, 

 72 [D loss: -0.016454633325338364] 
 72 [G loss: -0.019441716372966766, G obj.: 0.41393476724624634, G wgan.: -0.019441716372966766] 
 72 [real loss: -0.00593473669141531, fake loss: -0.026974529027938843]
 73 [D loss: -0.015315827913582325] 
 73 [G loss: -0.017069725319743156, G obj.: 0.40743017196655273, G wgan.: -0.017069725319743156] 
 73 [real loss: -0.003103075549006462, fake loss: -0.027528580278158188]
 74 [D loss: -0.01623412035405636] 
 74 [G loss: -0.017481151968240738, G obj.: 0.41097477078437805, G wgan.: -0.017481151968240738] 
 74 [real loss: -0.006284140516072512, fake loss: -0.026184100657701492]
 75 [D loss: -0.01568843424320221] 
 75 [G loss: -0.017817802727222443, G obj.: 0.39880305528640747, G wgan.: -0.017817802727222443] 
 75 [real loss: -0.005768908187747002, fake loss: -0.025607958436012268]
 76 [D loss: -0.017027389258146286] 
 76 [G loss: -0.014050071127712727, G obj.: 0.40201589465141296, G wgan.: -0.014050071127712727] 
 76 [real loss: -0.00723264506086707

C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


 101 [D loss: -0.015597844496369362] 
 101 [G loss: -0.02502981759607792, G obj.: 0.38889172673225403, G wgan.: -0.02502981759607792] 
 101 [real loss: -0.002605046145617962, fake loss: -0.028590641915798187]
 102 [D loss: -0.018020683899521828] 
 102 [G loss: -0.02816908434033394, G obj.: 0.40039265155792236, G wgan.: -0.02816908434033394] 
 102 [real loss: -0.008202281780540943, fake loss: -0.027839086949825287]
 103 [D loss: -0.02046743780374527] 
 103 [G loss: -0.02619701251387596, G obj.: 0.3980521857738495, G wgan.: -0.02619701251387596] 
 103 [real loss: -0.011817915365099907, fake loss: -0.029116960242390633]
 104 [D loss: -0.015912242233753204] 
 104 [G loss: -0.02819732204079628, G obj.: 0.4028922915458679, G wgan.: -0.02819732204079628] 
 104 [real loss: -0.0028448235243558884, fake loss: -0.02897966280579567]
 105 [D loss: -0.01838800311088562] 
 105 [G loss: -0.024587973952293396, G obj.: 0.3789111375808716, G wgan.: -0.024587973952293396] 
 105 [real loss: -0.008849071338

...

 641 [D loss: -0.0006598010659217834] 
 641 [G loss: -0.06574290990829468, G obj.: 0.26370665431022644, G wgan.: -0.06574290990829468] 
 641 [real loss: -0.06329610198736191, fake loss: 0.06197649985551834]
 642 [D loss: -0.001030033454298973] 
 642 [G loss: -0.06507620960474014, G obj.: 0.2684210240840912, G wgan.: -0.06507620960474014] 
 642 [real loss: -0.06040244176983833, fake loss: 0.05834237486124039]
 643 [D loss: -0.001216258853673935] 
 643 [G loss: -0.06798769533634186, G obj.: 0.258119136095047, G wgan.: -0.06798769533634186] 
 643 [real loss: -0.061185192316770554, fake loss: 0.058752674609422684]
 644 [D loss: -0.0024309568107128143] 
 644 [G loss: -0.06518331915140152, G obj.: 0.26379603147506714, G wgan.: -0.06518331915140152] 
 644 [real loss: -0.0623004324734211, fake loss: 0.05743851885199547]
 645 [D loss: -0.0017103888094425201] 
 645 [G loss: -0.06374317407608032, G obj.: 0.26288437843322754, G wgan.: -0.06374317407608032] 
 645 [real loss: -0.06301413476467133, f

 677 [D loss: -0.0008065495640039444] 
 677 [G loss: -0.06323175877332687, G obj.: 0.251001238822937, G wgan.: -0.06323175877332687] 
 677 [real loss: -0.05737345665693283, fake loss: 0.05576035752892494]
 678 [D loss: 0.0005486626178026199] 
 678 [G loss: -0.06497730314731598, G obj.: 0.2509205937385559, G wgan.: -0.06497730314731598] 
 678 [real loss: -0.05835356563329697, fake loss: 0.059450890868902206]
 679 [D loss: 0.0008624736219644547] 
 679 [G loss: -0.06490512937307358, G obj.: 0.24957415461540222, G wgan.: -0.06490512937307358] 
 679 [real loss: -0.05742839351296425, fake loss: 0.05915334075689316]
 680 [D loss: -0.0007248669862747192] 
 680 [G loss: -0.06667058169841766, G obj.: 0.24387550354003906, G wgan.: -0.06667058169841766] 
 680 [real loss: -0.057602252811193466, fake loss: 0.05615251883864403]
 681 [D loss: -0.00034285709261894226] 
 681 [G loss: -0.06430602818727493, G obj.: 0.25007641315460205, G wgan.: -0.06430602818727493] 
 681 [real loss: -0.0567595548927784, 

 713 [D loss: 0.0001662876456975937] 
 713 [G loss: -0.059119123965501785, G obj.: 0.26580020785331726, G wgan.: -0.059119123965501785] 
 713 [real loss: -0.050210461020469666, fake loss: 0.05054303631186485]
 714 [D loss: 0.0010254867374897003] 
 714 [G loss: -0.05737494304776192, G obj.: 0.26284143328666687, G wgan.: -0.05737494304776192] 
 714 [real loss: -0.050303444266319275, fake loss: 0.052354417741298676]
 715 [D loss: 0.00021339207887649536] 
 715 [G loss: -0.0584624819457531, G obj.: 0.25952398777008057, G wgan.: -0.0584624819457531] 
 715 [real loss: -0.05125720053911209, fake loss: 0.05168398469686508]
 716 [D loss: 9.914115071296692e-05] 
 716 [G loss: -0.05552980303764343, G obj.: 0.2616227865219116, G wgan.: -0.05552980303764343] 
 716 [real loss: -0.05083475261926651, fake loss: 0.051033034920692444]
 717 [D loss: 9.00905579328537e-05] 
 717 [G loss: -0.05689672753214836, G obj.: 0.2501693069934845, G wgan.: -0.05689672753214836] 
 717 [real loss: -0.050046406686306, fa

 749 [D loss: -0.001008765771985054] 
 749 [G loss: -0.05075075477361679, G obj.: 0.29337161779403687, G wgan.: -0.05075075477361679] 
 749 [real loss: -0.04601339250802994, fake loss: 0.04399586096405983]
 750 [D loss: -0.0005547888576984406] 
 750 [G loss: -0.05060138925909996, G obj.: 0.29059720039367676, G wgan.: -0.05060138925909996] 
 750 [real loss: -0.04257157817482948, fake loss: 0.0414620004594326]
 751 [D loss: -0.00045721419155597687] 
 751 [G loss: -0.0475958026945591, G obj.: 0.2841789722442627, G wgan.: -0.0475958026945591] 
 751 [real loss: -0.04438326507806778, fake loss: 0.043468836694955826]
 752 [D loss: -0.0005585327744483948] 
 752 [G loss: -0.05066753923892975, G obj.: 0.29149648547172546, G wgan.: -0.05066753923892975] 
 752 [real loss: -0.04543967545032501, fake loss: 0.04432260990142822]
 753 [D loss: 0.0001977682113647461] 
 753 [G loss: -0.049447864294052124, G obj.: 0.2959744334220886, G wgan.: -0.049447864294052124] 
 753 [real loss: -0.044510383158922195,

 785 [D loss: -0.0017394563183188438] 
 785 [G loss: -0.016158681362867355, G obj.: 0.3566177785396576, G wgan.: -0.016158681362867355] 
 785 [real loss: -0.023336024954915047, fake loss: 0.01985711231827736]
 786 [D loss: -0.003647654317319393] 
 786 [G loss: -0.013369519263505936, G obj.: 0.36749300360679626, G wgan.: -0.013369519263505936] 
 786 [real loss: -0.02715141884982586, fake loss: 0.019856110215187073]
 787 [D loss: -0.00672831991687417] 
 787 [G loss: -0.00824819691479206, G obj.: 0.36136797070503235, G wgan.: -0.00824819691479206] 
 787 [real loss: -0.028505336493253708, fake loss: 0.015048696659505367]
 788 [D loss: -0.007443695329129696] 
 788 [G loss: -0.013711679726839066, G obj.: 0.36493897438049316, G wgan.: -0.013711679726839066] 
 788 [real loss: -0.028516365215182304, fake loss: 0.013628974556922913]


In [13]:
def plot_training_curves(log, epoch, name, wgan=False):
    total_g_loss = np.array(log["g_loss"])[:, 0]
    total_d_loss = np.array(log["d_loss"])[:, 0] if not wgan else np.array(log["d_loss"])
    smoothed_tgl = src.smooth(np.array(log["g_loss"])[:, 0])
    smoothed_tdl = src.smooth(np.array(log["d_loss"])[:, 0]) if not wgan else src.smooth(np.array(log["d_loss"]))
    objective_loss = np.array(log["g_loss"])[:, 1]

    f, (a0, a1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [5, 2]})
    a0.plot(total_g_loss, alpha=0.3, c="b")
    a0.plot(smoothed_tgl, c="b", label="generator")
    a0.grid()
    if wgan:
        a0.plot(np.array(log["d_loss_real"]), c="g", label="real")
        a0.plot(np.array(log["d_loss_fake"]), c="r", label="fake")
    else:
        a0.plot(total_d_loss, alpha=0.3, c="orange")
        a0.plot(smoothed_tdl, c="orange", label="discriminator")
    a0.legend()
    a1.plot(objective_loss, alpha=0.9, c="green", label="L1 objective")
    a1.grid()
    a1.legend()
    f.text(0.5, 0, 'Iterations', ha='center', va='center')
    f.text(0, 0.5, 'Loss', ha='center', va='center', rotation='vertical')

    f.tight_layout()
    f.savefig(f"Plots/{name}_epoch_{epoch}_curves.png")

## Save features

Save model history

In [ ]:
np.save(name+"_log",log)

Save model weights

In [ ]:
combined.save_weights(name+"_model.h5")

Load model and predict

Predict future frames. Loads a 20 long sequence with 1000 sequence samples.

In [ ]:
sequence_test = src.load_datasets(prediction=True)

In [ ]:
sequence_test = src.augment_data(sequence_test[:100])

In [ ]:
sequence_test.shape

In [ ]:
#test
predictions = {}
past_frames = sequence_test[...,0:past]
test_truth = sequence_test[...,past:past+1]

In [ ]:
gen = combined.layers[1]

In [ ]:
for t in range(5):
    print(t)
    future = gen.predict(past_frames, batch_size=64)
    predictions[f"{t}"] = future
    past_frames = np.concatenate((past_frames[:,:,:,1:], predictions[f"{t}"]), axis=-1)
    test_truth = sequence_test[...,past+1+t:past+2+t]

Save example predictions

In [ ]:
src.save_examples(name, sequence_test, predictions, past, samples=[33,54]) # 33, 46, 54

Renormalize intensity values


In [ ]:
norms = np.load(sys.path[0]+"/5min_norms_compressed.npz")["arr_1"]

# *4 bc of augmentaion (it concats the frames so the 0th 1000th 2000th and 3000th are the same sample just rotated)
test_norms = list(norms[9000:])*4

#renormalize test samples
renormalized_test = np.array([sample * np.array(test_norms)[i] for i, sample in enumerate(sequence_test)])
renormalized_predictions = np.transpose((np.array([[sample * np.array(test_norms)[i] for i, sample in enumerate(predictions[key])] for key in ['0', '1', '2', '3']])[:,:,:,:,0]), (1,2,3,0))


Calculate pixel intensities back to dBZ and from there to mm/h. <br>
Sources: <br>
- https://www.dwd.de/DE/leistungen/radolan/radolan_info/radolan_radvor_op_komposit_format_pdf.pdf?__blob=publicationFile&v=11 (page 10)
- <https://web.archive.org/web/20160113151652/http://www.desktopdoppler.com/help/nws-nexrad.htm#rainfall%20rates>


In [ ]:
I_t.shape[-1]

In [ ]:
#dBZ
dBZ_t = renormalized_test*0.5 - 32.5
dBZ_p = renormalized_predictions*0.5 - 32.5
#mm/h
I_t = (0.005*10**(0.1*dBZ_t))**(0.625)
I_p = (0.005*10**(0.1*dBZ_p))**(0.625)

thresholds = [0.5]
scores = {}
for t in range(renormalized_predictions.shape[-1]): # loop over the predictions (4)
    print(t)
    for s in thresholds: # make a dict entry for each threshold score
        scores[f"pred_{t+1}"] = src.calculate_skill_scores(I_p[...,t:t+1],
                                                          I_t[...,past+t:past+1+t],
                                                          x=I_t[...,past-1:past],
                                                          threshold=s)

In [ ]:
np.save(name+"_scores",scores)

In [ ]:
np.mean((pd.Series(scores["pred_1"]["corr_to_truth"]).dropna()))

In [ ]:

np.mean((pd.Series(scores["pred_1"]["corr_to_input"]).dropna()))